## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [ ]:
# Mount from Azure Blob
dbutils.fs.mount(
  source = "wasbs://<container-name>@<storage-account-name>.blob.core.windows.net",
  mount_point = "/mnt/<mount-name>",
  extra_configs = {"<conf-key>":dbutils.secrets.get(scope = "<scope-name>", key = "<key-name>")})


--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-380574185150524> in <module> 
 3 source = "wasbs://<container-name>@<storage-account-name>.blob.core.windows.net" , 
 4 mount_point = "/mnt/<mount-name>" , 
 ----> 5 extra_configs = {"<conf-key>":dbutils.secrets.get(scope = "<scope-name>", key = "<key-name>")})
 
 /databricks/python_shell/dbruntime/dbutils.py in get (self, scope, key) 
 172 
 173 def get ( self , scope , key ) : 
 --> 174 return self . entry_point . getDbutils ( ) . preview ( ) . secret ( ) . get ( scope , key ) 
 175 
 176 def getBytes ( self , scope , key ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o376.get.
: java.lang.IllegalStateException: Secrets API is not enabled for this workspace.
	at com.databricks.backend.common.rpc.SimpleSecretManagerClient.performRequestReliably(SecretManagerClient.scala:121)
	at com.databricks.backend.common.rpc.SimpleSecretManagerClient.get(SecretManagerClient.scala:155)
	at com.databricks.backend.common.rpc.SimpleSecretManagerClient.getSecret(SecretManagerClient.scala:230)
	at com.databricks.dbutils_v1.impl.SecretUtilsImpl.getBytesInternal(SecretUtilsImpl.scala:46)
	at com.databricks.dbutils_v1.impl.SecretUtilsImpl.get(SecretUtilsImpl.scala:61)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [ ]:
# File location and type
file_location1 = "/mnt/<mount-file/Financial_Complaints_Product.csv"
file_location2 = "/mnt/<mount-file/Financial_Complaints_Issue.csv"
file_location3 = "/mnt/<mount-file/Financial_Complaints_Response.csv"
file_location4 = "/mnt/<mount-file/Financial_Complaints_Region.csv"
file_type = "csv"


# The applied options are for CSV files. For other file types, these will be ignored.
df1 = spark.read.csv(file_location1, header=True, inferSchema=True)
df2 = spark.read.csv(file_location2, header=True, inferSchema=True)
df3 = spark.read.csv(file_location3, header=True, inferSchema=True)
df4 = spark.read.csv(file_location4, header=True, inferSchema=True)

In [ ]:
df1.printSchema()

root
-- _c0: integer (nullable = true)
-- Complaint ID: integer (nullable = true)
-- Date Sumbited: string (nullable = true)
-- Product: string (nullable = true)
-- Sub-product: string (nullable = true)

In [ ]:
# create merged dataframe
from pyspark.sql.functions import col
df = df1.join(df2,["Complaint ID"]) \
     .join(df3,["Complaint ID"]) \
        .join(df4,["Complaint ID"])

In [ ]:
df.printSchema()

root
-- Complaint ID: integer (nullable = true)
-- _c0: integer (nullable = true)
-- Date Sumbited: string (nullable = true)
-- Product: string (nullable = true)
-- Sub-product: string (nullable = true)
-- _c0: integer (nullable = true)
-- Issue: string (nullable = true)
-- Sub-issue: string (nullable = true)
-- Company: string (nullable = true)
-- Company public response: string (nullable = true)
-- Company: string (nullable = true)
-- Consumer consent provided?: string (nullable = true)
-- Submitted via: string (nullable = true)
-- Tags: string (nullable = true)
-- Date Received: string (nullable = true)
-- Company response to consumer: string (nullable = true)
-- Timely response?: string (nullable = true)
-- Consumer disputed?: string (nullable = true)
-- _c0: integer (nullable = true)
-- State: string (nullable = true)
-- ZIP code: string (nullable = true)

In [ ]:
#conver string to date format
from pyspark.sql.functions import unix_timestamp, from_unixtime
df = df.select(
    'Date Sumbited', 
    from_unixtime(unix_timestamp('Date Sumbited', 'MM/dd/yyy')).alias('Date_Sumbited'), 'Date Received', 
    from_unixtime(unix_timestamp('Date Received', 'MM/dd/yyy')).alias('Date_Received'))

In [ ]:
#selected columns

column_list = ['Complaint ID', 'Date_Sumbited', 'Product', 'Sub-product', 'Issue',
       'Sub-issue', 'Company public response', 'State', 'ZIP code',
       'Tags', 'Consumer consent provided?', 'Submitted via', 'Date_Received',
       'Company response to consumer', 'Timely response?',
       'Consumer disputed?']
df = df.select([column for column in df.columns if column in column_list])  

In [ ]:
#Load to ADLS
df.write.csv("/mnt/<mount-name>"/Trans_Financial_Complaints, header = True, mode='overwrite')

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4046197382755029> in <module> 
 1 # 
 ----> 2 df . write . csv ( "/mnt/<mount-name>" / Trans_Financial_Complaints , header = True , mode = 'overwrite' ) 

 NameError : name 'Trans_Financial_Complaints' is not defined

In [ ]:
#try to create a delta table and save the data back to Azure blob.....so delta table appear in dafault table in data bricks but location on Azure blob
deltapath = "/mnt/<mount-name/deltadata"
df.write.format("delta").mode("append").option("path", deltapath).saveAsTable("default.mytableblob")
